# VGG16 cats vs dogs

# 两个教程的学习 1. 与 2.

## 1.学习up小土堆的视频

##### 1.1 导包

In [5]:
import time
import os
import torch
from PIL import Image
from matplotlib import pyplot as plt
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

##### 1.2 dataset的声明

In [ ]:
class MyData(Dataset):
    def __init__(self,root_dir,label_dir,transform=None):
            self.root_dir=root_dir
            self.label_dir=label_dir
            self.transform = transform
            
            self.path=os.path.join(self.root_dir,self.label_dir)
            self.img_list_path=os.listdir(self.path)
            #为了后面 CrossEntropyLoss() 函数期望的 label应该是整数型的张量
            self.class_idx = {'cats': 0, 'dogs': 1}  # 类别到索引的映射
            

    def convert_rgba_to_rgb(self, image):
        image_rgb = Image.new("RGB", image.size)
        image_rgb.paste(image)
        return image_rgb


    def __getitem__(self,idx):
       img_name=self.img_list_path[idx]
       self.path = os.path.join(self.root_dir, self.label_dir,img_name)
        
       image = Image.open(self.path)  #读取这个idx对应的图片
       if image.mode != 'RGB':
           image = self.convert_rgba_to_rgb(image)  # Convert image to RGB if it's in RGBA format
       label=self.label_dir           #并且此图片的标签就是“传”过来的 train文件 下的 子文件名称
       label = self.class_idx[label]  # 将标签映射为整数
     

       
       image=self.transform(image)
       return  image,label

    def __len__(self):
       return len(self.img_list_path)

##### 1.3 transform

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 假设使用224x224大小的图片输入
    transforms.ToTensor(),           # 将图片转换为张量
])

##### 1.4 dataset 的定义

In [ ]:
root_dir_train = "dataset/training_set"
root_dir_test = "dataset/test_set"

cats_dataset=MyData(root_dir_train,"cats",transform=transform)
dogs_dataset=MyData(root_dir_train,"dogs",transform=transform)
cats_test_dataset = MyData(root_dir_test, "cats", transform=transform)
dogs_test_dataset = MyData(root_dir_test, "dogs", transform=transform)
print(f"猫训练数据集有{len(cats_dataset)}张，猫测试数据集有{len(cats_test_dataset)}张")
print(f"狗训练数据集有{len(dogs_dataset)}张，狗测试数据集有{len(dogs_test_dataset)}张")

# 训练数据集
train_dataset= cats_dataset + dogs_dataset

# 测试数据集
total_test_dataset = cats_test_dataset + dogs_test_dataset

print(f"训练数据集有{len(train_dataset)}张，测试数据集有{len(total_test_dataset)}张")

##### 1.5 dataloader

In [ ]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=0)

total_test_dataloader=DataLoader(total_test_dataset,batch_size=batch_size,shuffle=True,num_workers=0)

##### 1.6 device (GPU能加速 1网络模型、2损失函数、3数据集的图和label)

In [ ]:
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

print("使用的设备{}".format(device))

##### 1.7.1  net (下面这个NET1() 类是跟着视频敲的)

In [ ]:
class NET1(nn.Module):

    def __init__(self):
        super().__init__()
        # 第一个卷积块
        self.conv1=nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1=nn.ReLU(inplace=True)

        self.conv2=nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU(inplace=True)

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 第二个卷积块
        self.conv3=nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU(inplace=True)

        self.conv4=nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU(inplace=True)

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 第三个卷积块
        self.conv5=nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU(inplace=True)

        self.conv6=nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu6 = nn.ReLU(inplace=True)

        self.conv7=nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu7 = nn.ReLU(inplace=True)

        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 第四个卷积块
        self.conv8=nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu8 = nn.ReLU(inplace=True)

        self.conv9=nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu9 = nn.ReLU(inplace=True)

        self.conv10=nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu10 = nn.ReLU(inplace=True)

        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        # 第五个卷积块
        self.conv11=nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu11 = nn.ReLU(inplace=True)

        self.conv12=nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu12 = nn.ReLU(inplace=True)

        self.conv13=nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu13 = nn.ReLU(inplace=True)

        self.pool5=nn.MaxPool2d(kernel_size=2, stride=2)

    # flatten(x, 1)
        self.flatten=nn.Flatten()

    #三个全连接层
        self.linear1=nn.Linear(512 * 7 * 7, 4096)
        self.relu14 = nn.ReLU(inplace=True)
        self.dropout=nn.Dropout()

        self.linear2=nn.Linear(4096, 4096)
        self.relu15 = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout()

        self.linear3=nn.Linear(4096, 2)

    def forward(self, x):

       x = self.conv1(x)
       x = self.relu1(x)
       x = self.conv2(x)
       x = self.relu2(x)
       x = self.pool1(x)

       x = self.conv3(x)
       x = self.relu3(x)
       x = self.conv4(x)
       x = self.relu4(x)
       x = self.pool2(x)

       x = self.conv5(x)
       x = self.relu5(x)
       x = self.conv6(x)
       x = self.relu6(x)
       x = self.conv7(x)
       x = self.relu7(x)
       x = self.pool3(x)

       x = self.conv8(x)
       x = self.relu8(x)
       x = self.conv9(x)
       x = self.relu9(x)
       x = self.conv10(x)
       x = self.relu10(x)
       x = self.pool4(x)

       x = self.conv11(x)
       x = self.relu11(x)
       x = self.conv12(x)
       x = self.relu12(x)
       x = self.conv13(x)
       x = self.relu13(x)
       x =  self.pool5(x)
       #
       x = self.flatten(x)
       #
       x = self.linear1(x)
       x = self.relu14(x)
       x = self.dropout()

       x = self.linear2(x)
       x = self.relu15(x)
       x = self.dropout()

       x = self.linear3(x)


       return x


torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

### Parameters:
 in_channels (int) – Number of channels in the input image

 out_channels (int) – Number of channels produced by the convolution

 kernel_size (int or tuple) – Size of the convolving kernel

stride (int or tuple, optional) – Stride of the convolution. Default: 1

 padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0

padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'

dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1

groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1

bias (bool, optional) – If True, adds a learnable bias to the output. Default: True

##### 1.7.2  下面的NET2类使用了nn.Sequential( )

In [ ]:
class NET2(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            # 第一个卷积块
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # CON1 in_channel=3,out_channel=64  (3,224,224)->(64,224,224)
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), #CON2 in_channel=64,out_channel=64 (64,224,224)->(64,224,224)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Pool_1  (64,  [224,224])->(64,  [112,112])

            # 第二个卷积块
            nn.Conv2d(64, 128, kernel_size=3, padding=1),   #CON3 out_channel=128  (64,112,112)->( [128],112,112)
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),   #CON4 out_channel=128  (128,112,112)->( 128 ,112,112)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Pool_2  (128,   [112,112])->(128,   [56,56])

            # 第三个卷积块
            nn.Conv2d(128, 256, kernel_size=3, padding=1), #CON5 out_channel=256  (128,56,56)->( [256] ,56,56)
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), #CON6 (256,56,56)->(256,56,56)
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), #CON7 (256,56,56)->(256,56,56)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),       # Pool_3  (256, [56,56])->(256, [28,28])

            # 第四个卷积块
            nn.Conv2d(256, 512, kernel_size=3, padding=1), #CON8 (256,28,28)->( 512 ,28,28)
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), #CON9 (512,28,28)->( 512 ,28,28)
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), #CON10 (512,28,28)->( 512 ,28,28)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),      # Pool_4  (512, [28,28])->(512, [14,14])

            # 第五个卷积块
            nn.Conv2d(512, 512, kernel_size=3, padding=1), #CON11 (512,14,14)->( 512 ,14,14)
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), #CON12 (512,14,14)->( 512 ,14,14)
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1), #CON13 (512,14,14)->( 512 ,14,14)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),     # Pool_5  (512, [14,14])->(512, [7,7])

        )


         ########################   然后执行   flatten(x, 1)

        # 定义VGG16的全连接部分，包括三个全连接层
        self.classifier = nn.Sequential(
            nn.Linear(25088, 4096),  # 512 * 7 * 7 -> 4096*1*1
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),         # 4096*1*1 -> 4096*1*1
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 1000),   # 4096*1*1 -> 2*1*1
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  #############  flatten(x, 1)
        x = self.classifier(x)
        return x



if __name__=='__main__':
    in_data=torch.ones(1,3,224,224)
    net = NET2()
    net.load_state_dict(torch.load(r"vgg16-397923af.pth"))
    net.classifier.add_module("my",nn.Linear(1000,2))
    print(net)
    out= net (in_data)
    print(out)

##### 1.8 把net放到device

In [ ]:
net = net.to(device)

##### 1.9损失函数

In [ ]:
from torch.nn import CrossEntropyLoss
loss=CrossEntropyLoss()

##### 1.10 损失函数放到device

In [ ]:
loss = loss.to(device)

##### 1.11优化器

In [ ]:
learn_rate= 1e-4

# opt=torch.optim.Adam(net.parameters(),lr=learn_rate)
optim=torch.optim.SGD(net.parameters(),lr=learn_rate)

##### 1.12训练过程和验证过程 框架

In [ ]:
list_train_losses = []
list_test_losses = []

# train
for i in range(2):
          print("-------第 {} 轮训练开始-------".format(i+1))
          start_time=time.time()

          loss_in_one_epoch=0.0
          accuracy_sum=0.0

          for data in train_dataloader:
              images, labels = data
              images = images.to(device)
              labels = labels.to(device)

              optim.zero_grad()  # 每次都不被上一次的权重影响

              outputs = net(images)
              loss_out = loss(outputs, labels)

              loss_out.backward()
              optim.step()  #调优

              loss_in_one_epoch += loss_out
              list_train_losses.append(loss_out)
              accuracy = (outputs.argmax(1) == labels).sum()
              accuracy_sum += accuracy
          end_time=time.time()
          print("第{}轮训练结束,用时{},总的loss为{}".format(i + 1,end_time-start_time,loss_in_one_epoch))
          print("这次的正确率: {}".format(accuracy_sum/len(train_dataset)))

          


          # 测试
          sum_loss = 0.0
          accuracy_sum = 0.0
          for images, labels in total_test_dataloader:
              with torch.no_grad():
                  images = images.to(device)
                  labels = labels.to(device)

                  optim.zero_grad()  ########################新增 梯度清零
                  outputs = net(images)

                  accuracy = (outputs.argmax(1) == labels).sum()
                  accuracy_sum += accuracy
                  loss_1 = loss(outputs, labels)
                  sum_loss += loss_1
                  list_test_losses.append(loss_1)
          print("这一次测试的loss是{}".format(sum_loss))
          print("此次测试的正确率: {}".format(accuracy_sum / len(total_test_dataset)))


          torch.save(net, "net_{}.pth".format(i+1))
          print("模型已保存")


##### 1.13测试过程

In [ ]:
image = Image.open(output_image_path)
test = transform(image)
test = torch.reshape(test, (1, 3, 224, 224))
test = test.to('cuda')


model = torch.load(r"net_2.pth")
output = model(test)

a=int(output.argmax(1))

if a == 1:  # 标签的数字索引1对应dog
  print("dog")
else:
  print("cat")

plt.imshow(image)
plt.show()

# 2. 学习别人的VGG16 CATS vs DOGS,然后升级自己的成为4分类

#### 与我上面所学习的不同之处是：
####                                                   .1数据集的处理，
####                                                   .2VGG网络的构建方式
####                                                   .3测试的交互界面更好

##### 2.1 导包

In [ ]:
import torch
from numpy import random
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from matplotlib import pyplot as plt
import torch.nn.functional

##### 2.2生成一个.txt（.txt的每行 是 图片地址 和 这张图片的label ）

In [ ]:
import os

animal_class= ['cat','dog','bee','ant']  # "train" 有新增了 bee , ant
root_dir=r"train"

if __name__=='__main__':
    f=open("data.txt",mode="w")
    items = os.listdir(root_dir) #返回一个包含此目录中所有子目录名称的列表list
    for item in items:           #for循环去访问list中的每个元素

        if item not in animal_class:
            continue

        # label
        label=animal_class.index(item)
        #图片名字
        pic_path = os.path.join(root_dir,item)
        pic_name_list=os.listdir(pic_path) #返回一个包含此目录下所有图片名称的列表
        # 写入txt
        for every_pic_name in pic_name_list:
           f.write(str(label)+";"+"%s%s%s" % (pic_path,'/',every_pic_name))
           f.write("\n")

    f.close()

# 很关键的os.listdir（）函数，能够或得你输入的path下面所有文件的名字的列表

##### 2.3定义mydataset类

In [ ]:
class mydataset(Dataset):
    """
    这是一个继承了Dataset类的子类，
    需要去传入你的包含有图片名称的list以及transform，
    list可以是通过切片的一段sub_list，这样可以有不同的sub_list对应train_dataset和 test_dataset
    """
    def __init__(self,sub_list,transform=None):
        self.sub_list=sub_list
        self.transform=transform

    def __len__(self):
        return len(self.sub_list)

    def __getitem__(self, index):
        #标签
        label=int(self.sub_list[index].split(";")[0])

        #图片
        image_path=self.sub_list[index].split(";")[1].strip('\n')
        image=Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            image.save(image_path)

        #transform
        image=self.transform(image)

        return image,label

##### 2.4 dataset

In [ ]:
# 打开txt.py得到的“data.txt”,用readlines方法得到一个list
with open("data.txt", "r") as f:
    lines = f.readlines()


# 打乱lines的顺序,为数据集做准备
random.shuffle(lines)

val_number = int(len(lines) * 0.2)
train_number = len(lines) - val_number


# transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 假设使用224x224大小的图片输入
    transforms.ToTensor(),  # 将图片转换为张量
])

# dataset
val_dataset = mydataset(lines[:val_number], transform=transform)
train_dataset = mydataset(lines[val_number:], transform=transform)
print(f"train集长度{len(train_dataset)},val集的长度{len(val_dataset)}")

##### 2.5 dataloader

In [ ]:
# DataLoader 注意 "D"和"L"都是大写
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

##### 2.6device

In [ ]:
# device
device = torch.device("cuda:0")

##### 2.7.1 net VGG16  
*从pytorch 官网的VGG16源码里面来的，其实也可以自己去构建*

In [ ]:
model_urls = {
    "vgg16": "https://download.pytorch.org/models/vgg16-397923af.pth",

}#权重下载网址


class VGG(nn.Module):
    def __init__(self, features, num_classes = 1000, init_weights= True, dropout = 0.5):
        super(VGG,self).__init__()
        self.features = features   # features参数表示卷积层的结构 ？？？
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))#AdaptiveAvgPool2d使处于不同大小的图片也能进行分类
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),#完成4096的全连接
            nn.Linear(4096, num_classes),#对num_classes的分类
        )
        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm2d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)  #卷积
        x = self.avgpool(x)   #池化
        x = torch.flatten(x, 1)#对输入层进行平铺，转化为一维数据
        x = self.classifier(x) #全连接
        return x


def make_layers(cfg, batch_norm = False):#make_layers对输入的cfg进行循环
    layers = []
    in_channels = 3
    for v in cfg:#对cfg进行输入循环,取第一个v
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]#把输入图像进行缩小
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)#输入通道是3，输出通道64
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],

}

def vgg16(pretrained=False, progress=True,num_classes=2):
    model = VGG(make_layers(cfgs['D']))
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['vgg16'],model_dir='./model' ,progress=progress)#预训练模型地址
        model.load_state_dict(state_dict)
    if num_classes !=1000:
        model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),#随机删除一部分不合格
            nn.Linear(4096, 4096),
            nn.ReLU(True),#防止过拟合
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
    return model



if __name__=='__main__':
    in_data=torch.ones(1,3,224,224)
    net=vgg16(pretrained=True, progress=True,num_classes=2)
    print(net)
    out=net(in_data)
    print(out)

##### 2.7.2 自己去构建NET类

In [ ]:
class NET(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, padding=1, kernel_size=3),
            nn.ReLU(),  # 参数inplace = True是指原地进行操作，操作完成后覆盖原来的变量

            nn.Conv2d(in_channels=64, out_channels=64, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),  # 224*224 变成了 112*112

            nn.Conv2d(in_channels=64, out_channels=128, padding=1, kernel_size=3),
            nn.ReLU(),  # 参数inplace = True是指原地进行操作，操作完成后覆盖原来的变量

            nn.Conv2d(in_channels=128, out_channels=128, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),  # 112*112变成56*56

            nn.Conv2d(in_channels=128, out_channels=256, padding=1, kernel_size=3),
            nn.ReLU(),  # 参数inplace = True是指原地进行操作，操作完成后覆盖原来的变量

            nn.Conv2d(in_channels=256, out_channels=256, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(in_channels=256, out_channels=256, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),  # 56*56变成 28*28

            nn.Conv2d(in_channels=256, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),  # 28*28 变成 14*14

            nn.Conv2d(in_channels=512, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, padding=1, kernel_size=3),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),  # 14*14 变成 7*7
        )

        self.classifier = nn.Sequential(
            nn.Linear(25088, 4096),  # 512 * 7 * 7 -> 4096*1*1
            nn.ReLU(True),
            nn.Dropout(),  # 防止过拟合

            nn.Linear(4096, 4096),  # 4096*1*1 -> 4096*1*1
            nn.ReLU(True),
            nn.Dropout(),  # 防止过拟合

            nn.Linear(4096, 1000),
        )

    def forward(self, x):
        x = self.features(x)
        x = flatten(x, 1)
        x = self.classifier(x)
        return x

##### 2.7.3 加载2.7.2的NET类

In [ ]:
net=NET()
net.load_state_dict(torch.load(r"vgg16-397923af.pth"))
net.classifier.add_module("my",nn.Linear(1000,4)) # 注意我这里最后是4，因为4分类。

net.to(device)

##### 2.8 其他

In [ ]:
# opt优化器
optim = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
# 损失函数
my_loss = nn.CrossEntropyLoss()

In [ ]:
# TRAIN & VAL
epochs = int(input('输入轮数：'))

list_train_losses = []
list_test_losses = []

for epoch in range(epochs):

    train_losses = 0.0
    test_losses = 0.0
    total_accuracy = 0.0

    for data in train_dataloader:
        image, label = data
        image = image.to(device)
        label = label.to(device)

        optim.zero_grad()
        output = net(image)

        train_loss = my_loss(output, label)
        train_loss.backward()
        optim.step()  # 优化器更新
        train_losses += train_loss

    list_train_losses.append(train_losses / len(train_dataset))  # 平均损失相加
    # 测试
    for data in val_dataloader:
        image, label = data
        with torch.no_grad():
            image = image.to(device)
            label = label.to(device)

            out = net(image)  # 投入网络

            test_loss = my_loss(out, label)
            test_losses += test_loss

            accuracy = (out.argmax(1) == label).sum()  #####
            # accuracy = ((out.argmax(1) == label).sum()).clone().detach().cpu().numpy()  # 正确预测的总和比测试集的长度，即预测正确的精度
            total_accuracy += accuracy

    list_test_losses.append(test_losses / len(val_dataset))

    print("第{}轮".format(epoch + 1))
    print("训练集损失：{}".format(train_losses))
    print("验证集损失：{}".format(test_losses))
    print("验证集上的精度：{:.1%}".format(total_accuracy / len(val_dataset)))

    # torch.save(net,"4_classes_{}.pth".format(epoch+1))
    # print("模型已保存")

# list_train_losses = [item.item() for item in list_train_losses]
# list_test_losses = [item.item() for item in list_test_losses]
#
# plt.plot(list(range(1, len(list_train_losses) + 1)), list_train_losses, color='blue', marker='o', label='train_loss')
# plt.plot(list(range(1, len(list_test_losses) + 1)), list_test_losses, color='red', marker='o', label='test_loss')
#
# plt.legend()
# plt.grid(True)
# plt.savefig(r"plot.png")
# plt.close()

##### 2.9 test.py

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 假设使用224x224大小的图片输入
    transforms.ToTensor(),  # 将图片转换为张量
])



image_path=r"test.png"

i=Image.open(image_path)
if i.mode != 'RGB':
    i = i.convert('RGB')
    i.save(image_path)

img=Image.open(image_path)
image=transform(img)
image = torch.reshape(image, (1, 3, 224, 224))

model=torch.load(r"4_classes_1.pth")
image=image.to('cuda')


out = model(image)

print("从模型出来的out",out)
out=torch.nn.functional.softmax(out,dim=1)
print("经过softmax的out",out)

a=int(out.argmax(1))

out=out.data.cpu().numpy()
# animal_class=['cat','dog','bee','ant']
plt.suptitle("Classes:{},P={:.1%}".format(animal_class[a],out[0,a]))
plt.imshow(img)
plt.show()